In [1]:
from contextlib import contextmanager
import sqlite3
import yaml

In [2]:
! pwd

/home/jeremy/projects/hyperborea3/scripts/spells


In [3]:
load_folder = "lvl3"

In [4]:
@contextmanager
def db_cur():
    URI = f"../../hyperborea3/hyperborea.sqlite3"
    con = sqlite3.connect(URI, check_same_thread=False)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    yield cur
    con.commit()
    con.close()

Test connection to db

In [5]:
with db_cur() as cur:
    cur.execute("""
        SELECT *
          FROM spells;
    """)
    result = [dict(x) for x in cur.fetchall()]
result[:1]

[{'spell_id': 1,
  'spell_name': 'Acid Arrow',
  'rng': '30 feet',
  'dur': 'special',
  'reversible': 0,
  'pp': 175,
  'spell_desc': '<p>A magic arrow darts from the finger of the caster. On a successful attack roll (<i>dexterity</i> modifier applies), the <i>acid arrow</i> strikes for 1d4+1 hp physical damage, plus an additional 2d4 hp acid damage in the same round. Magicians (but not other sorcerers) enjoy a +1 bonus to the attack roll for every 2 CA levels (CA 3–4 = +2, CA 5–6 = +3, and so forth). Acid damage will persist for higher level sorcerers:</p> <ul><li>1 extra round for CA 4–6</li> <li>2 extra rounds for CA 7–9</li> <li>3 extra rounds for CA 10 or greater.</li></ul> <p>For example, an <i>acid arrow</i> fired by a CA 12 sorcerer on round 1 would inflict 1d4+1 hp base damage plus 2d4 hp acid damage on round 1, 2d4 hp acid damage on round 2, 2d4 hp acid damage on round 3, and a final 2d4 hp acid damage on round 4. The acid may ruin armour or clothing per referee discretion. 

In [6]:
files = ! ls {load_folder}/*.yml
files

['lvl3/Agonizing_Touch.yml',
 'lvl3/Allay_Exhaustion.yml',
 'lvl3/Animate_Carrion_II.yml',
 'lvl3/Animate_Dead.yml',
 'lvl3/Blinding_Light.yml',
 'lvl3/Blink.yml',
 'lvl3/Call_Lightning.yml',
 'lvl3/Cataleptic_State.yml',
 'lvl3/Clairaudience.yml',
 'lvl3/Clairvoyance.yml',
 'lvl3/Cold_Protection.yml',
 'lvl3/Continuous_Darkness.yml',
 'lvl3/Create_Food_and_Water.yml',
 'lvl3/Cryonic_State.yml',
 'lvl3/Cure_Blindness.yml',
 'lvl3/Cure_Deafness.yml',
 'lvl3/Cure_Disease.yml',
 'lvl3/Death_Masque.yml',
 'lvl3/Deceive.yml',
 'lvl3/Dispel_Magic.yml',
 'lvl3/Dispel_Phantasm.yml',
 'lvl3/Dissipate_Gas.yml',
 'lvl3/Exploding_Skull.yml',
 'lvl3/Explosive_Runes.yml',
 'lvl3/Fear.yml',
 'lvl3/Fire_Protection.yml',
 'lvl3/Fire_Staff.yml',
 'lvl3/Fireball.yml',
 'lvl3/Flame_Arrow.yml',
 'lvl3/Floating_Skull.yml',
 'lvl3/Fly.yml',
 'lvl3/Freeze_Surface.yml',
 'lvl3/Glyph_of_Warding.yml',
 'lvl3/Hallucinatory_Terrain.yml',
 'lvl3/Haste.yml',
 'lvl3/Hold_Animal.yml',
 'lvl3/Illusory_Script.yml',
 'lv

In [7]:
def load_yml_spell(file_name):
    with open(file_name, "r") as f:
        spell = yaml.safe_load(f)
    return spell

In [8]:
def spell_preload_check(spell):
    with db_cur() as cur:
        cur.execute(
            """
            SELECT *
            FROM spells
            WHERE spell_id = ?;
            """,
            (spell["id"],),
        )
        db_spell = cur.fetchone()
    assert spell["name"] == db_spell["spell_name"]
    assert isinstance(spell["id"], int)
    assert isinstance(spell["pp"], int)
    assert isinstance(spell["Rng"], (str, int))
    assert isinstance(spell["Dur"], str)
    assert spell["Rev"] in [0, 1]
    assert isinstance(spell["Desc"], str)
    assert spell["Desc"][:3] == "<p>"
    if spell["id"] not in [53]:
        assert spell["Desc"][-4:] == "</p>"



In [9]:
def update_spell(spell):
    with db_cur() as cur:
        cur.execute(
            """
            UPDATE spells
            SET rng = ?
                , dur = ?
                , reversible = ?
                , pp = ?
                , spell_desc = ?
            WHERE spell_id = ?;
            """,
            (
                spell["Rng"],
                spell["Dur"],
                spell["Rev"],
                spell["pp"],
                spell["Desc"],
                spell["id"],
            ),
        )
    return 

### Test parsing all the files

In [11]:
for f in files:
    print(f"checking {f}")
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    print("success")
    print()

checking lvl3/Agonizing_Touch.yml
success

checking lvl3/Allay_Exhaustion.yml
success

checking lvl3/Animate_Carrion_II.yml
success

checking lvl3/Animate_Dead.yml
success

checking lvl3/Blinding_Light.yml
success

checking lvl3/Blink.yml
success

checking lvl3/Call_Lightning.yml
success

checking lvl3/Cataleptic_State.yml
success

checking lvl3/Clairaudience.yml
success

checking lvl3/Clairvoyance.yml
success

checking lvl3/Cold_Protection.yml
success

checking lvl3/Continuous_Darkness.yml
success

checking lvl3/Create_Food_and_Water.yml
success

checking lvl3/Cryonic_State.yml
success

checking lvl3/Cure_Blindness.yml
success

checking lvl3/Cure_Deafness.yml
success

checking lvl3/Cure_Disease.yml
success

checking lvl3/Death_Masque.yml
success

checking lvl3/Deceive.yml
success

checking lvl3/Dispel_Magic.yml
success

checking lvl3/Dispel_Phantasm.yml
success

checking lvl3/Dissipate_Gas.yml
success

checking lvl3/Exploding_Skull.yml
success

checking lvl3/Explosive_Runes.yml
succes

Do it for real

In [12]:
for f in files:
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    update_spell(spell)

In [13]:
load_yml_spell(files[-1])

{'id': 452,
 'name': 'Wraithshape',
 'pp': 246,
 'Lvl': ['ill 3', 'nec 3', 'wch 3'],
 'Rng': 0,
 'Dur': '1 turn per CA level',
 'Rev': 0,
 'Desc': '<p>The sorcerer and all gear assume a smoke-grey, incorporeal form. Whilst in <i>wraithshape</i>, the undead (except <b>vampires</b> and <b>liches</b>) will ignore the caster, believing him or her to be one of their own abysmal kind. The sorcerer can squeeze under doors or through small holes or cracks, and he can float in the air at 15 MV. Only magical weapons or spells can harm such a sorcerer. The sorcerer cannot, however, make any attacks unless confronted by a creature in a like state. <i>Dispel magic</i> breaks the spell; otherwise, the sorcerer can terminate it at will.</p>'}